<a href="https://colab.research.google.com/github/maribardella/agente-especificar-user-story/blob/main/Agente_de_Especifica%C3%A7%C3%A3o_de_User_Stories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip -q install google-genai

In [4]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [10]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [15]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [16]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [17]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [79]:
##########################################
# --- Agente 1: Analisador de Contexto --- #
##########################################
def agente_analisador_contexto(contexto):

    analisador_contexto = Agent(
        name="agente_analisador_contexto",
        model="gemini-2.0-flash",
        instruction="""
        Você é um analista de produto altamente experiente, com profundo conhecimento em elicitação de requisitos e compreensão de necessidades de negócio.
        Seu objetivo é interpretar o input fornecido por um Product Owner e extrair os elementos essenciais para criação de uma user story de qualidade.
        Utilize boas práticas de análise de requisitos para identificar:
        Quem é o usuário ou persona envolvida?
        Qual é a ação ou funcionalidade desejada?
        Qual é o benefício ou valor que essa ação traz para o usuário?
        Há telas, fluxos ou sistemas mencionados?
        Há informações ambíguas ou faltantes que precisam de esclarecimento?
        """,
        description="Agente que faz o entendimento sobre o produto, personas e funcionalidades.",
    )

    entrada_do_agente_analisador_contexto = f"Contexto: {contexto}"

    contexto_analisado = call_agent(analisador_contexto, entrada_do_agente_analisador_contexto)
    return contexto_analisado

In [75]:
################################################
# --- Agente 2: Gerador de User Stories --- #
################################################
def agente_gerador_user_stories(contexto, contexto_analisado):
    gerador_user_stories = Agent(
        name="agente_gerador_user_stories",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista de produtos e sabe especificar demandas como ninguém.
        Com base nas boas práticas de Product Management e metodologias ágeis, sua missão é redigir user stories claras, concisas e centradas no usuário.
        Utilize o formato padrão:
        Como [persona], quero [ação], para que [benefício]
        Certifique-se de que:
        A persona esteja clara e contextualizada
        A ação seja específica e objetiva
        O benefício represente o valor ou motivação real por trás da ação
        """,
        description="Agente que Produz as user stories no formato padrão",
    )

    entrada_do_agente_gerador_user_stories = f"Contexto:{contexto}\nContexto analisado: {contexto_analisado}"
    # Executa o agente
    user_stories = call_agent(gerador_user_stories, entrada_do_agente_gerador_user_stories)
    return user_stories

In [69]:
######################################
# --- Agente 3: Gerador de Critérios de Aceitação --- #
######################################
def agente_gerador_criterios(contexto, contexto_analisado, user_stories):
    gerador_criterios = Agent(
        name="agente_gerador_criterios",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Product Manager especializado em documentação de requisitos para times ágeis.
        Seu objetivo é transformar uma user story em critérios de aceitação claros, objetivos e testáveis, garantindo que o time de desenvolvimento e QA saiba exatamente quando considerar a entrega como "pronta".
        Siga estas boas práticas:
        Escreva em linguagem objetiva
        Evite ambiguidade ou termos vagos
        Use a estrutura: "O sistema deve..." ou "Deve ser possível..."
        Inclua critérios positivos e negativos, se aplicável
        """,
        description="Constrói critérios de aceitação claros, verificáveis e objetivos."
    )
    entrada_do_agente_gerador_criterios = f"Contexto: {contexto}\nContexto analisado: {contexto_analisado}\nUser stories: {user_stories}"
    # Executa o agente
    criterios = call_agent(gerador_criterios, entrada_do_agente_gerador_criterios)
    return criterios

In [77]:
##########################################
# --- Agente 4: Revisor Gerador de Cenário de Teste --- #
##########################################
def agente_gerador_cenario(contexto, contexto_analisado, user_stories,criterios):
    gerador_cenario = Agent(
        name="agente_gerador_cenario",
        model="gemini-2.0-flash",
        instruction="""
        Você é um analista de QA com foco em automação e comportamento orientado a testes (BDD).
        Sua missão é gerar cenários de teste comportamentais (Given / When / Then) com base na user story e nos critérios de aceitação fornecidos.
        Boas práticas que devem ser seguidas:
        Cada critério deve gerar pelo menos 1 cenário
        Use o formato Gherkin: Cenario, Dado que, Quando, Então
        Crie títulos de cenário descritivos
        Explore variações positivas e negativas, se possível
        Formate o texto de uma forma visual
        """,
        description="traduz os critérios de aceitação em cenários no estilo BDD (Behavior-Driven Development)"
    )
    entrada_do_agente_gerador_cenario = f"Contexto: {contexto}\nContexto analisado: {contexto_analisado}\nUser stories: {user_stories}\nCriterios: {criterios}"
    # Executa o agente
    cenarios = call_agent(gerador_cenario, entrada_do_agente_gerador_cenario)
    return cenarios

In [80]:
print("🚀 Iniciando o Sistema de Especificação de User Stories com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
contexto = input("❓ Por favor, digite o CONTEXTO sobre o qual você quer especificar seus cards: ")

# Inserir lógica do sistema de agentes ################################################
if not contexto:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {contexto}")

    contexto_analisado = agente_analisador_contexto(contexto)
    print("\n--- 📝 Resultado do Agente 1 (Analisador de Contexto) ---\n")
    display(to_markdown(contexto_analisado))
    print("--------------------------------------------------------------")

    user_stories = agente_gerador_user_stories(contexto, contexto_analisado)
    print("\n--- 📝 Resultado do Agente 2 (Gerador de User Stories) ---\n")
    display(to_markdown(user_stories))
    print("--------------------------------------------------------------")

    criterios = agente_gerador_criterios(contexto, contexto_analisado, user_stories)
    print("\n--- 📝 Resultado do Agente 3 (Gerador de Critérios de Aceitação) ---\n")
    display(to_markdown(criterios))
    print("--------------------------------------------------------------")

    cenarios = agente_gerador_cenario(contexto, contexto_analisado, user_stories,criterios)
    print("\n--- 📝 Resultado do Agente 4 (Gerador de Cenários de Teste) ---\n")
    display(to_markdown(cenarios))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Especificação de User Stories com 4 Agentes 🚀
❓ Por favor, digite o CONTEXTO sobre o qual você quer especificar seus cards: um funcionario do rh de uma escola que precisa lançar a quantidade de aulas semanais que cada professor ministra e isso é usado para fazer os cálculos do pagamento do salário do professor
Maravilha! Vamos então criar o post sobre novidades em um funcionario do rh de uma escola que precisa lançar a quantidade de aulas semanais que cada professor ministra e isso é usado para fazer os cálculos do pagamento do salário do professor

--- 📝 Resultado do Agente 1 (Analisador de Contexto) ---



> Compreendo o contexto. O sistema precisa permitir que o funcionário do RH da escola registre a quantidade de aulas semanais ministradas por cada professor, informação essencial para o cálculo correto da folha de pagamento.
> 
> **Análise detalhada:**
> 
> *   **Usuário/Persona:** Funcionário do RH da escola.
> *   **Ação:** Lançar/registrar a quantidade de aulas semanais ministradas por um professor.
> *   **Benefício:**
>     *   Para o RH: Garantir a precisão no cálculo da folha de pagamento dos professores, evitando erros e retrabalho.
>     *   Para a escola: Manter a conformidade com as obrigações trabalhistas e garantir a satisfação dos professores com o pagamento correto.
> *   **Possíveis telas/fluxos:**
>     *   Tela de listagem de professores.
>     *   Tela de detalhe do professor, com campo para inserir a quantidade de aulas semanais.
>     *   Possível tela de resumo/validação dos dados inseridos.
> *   **Sistemas envolvidos:**
>     *   Sistema de gestão de RH/folha de pagamento da escola.
> *   **Informações ambíguas/faltantes:**
>     *   Como o sistema identifica os professores? (ID, nome, etc.)
>     *   Como o sistema valida a quantidade de aulas inserida? (Existe um limite máximo? Há algum tipo de regra de negócio?)
>     *   O sistema permite registrar aulas extras ou substituições?
>     *   O sistema mantém um histórico das quantidades de aulas lançadas?
>     *   Como essa informação é utilizada no cálculo da folha de pagamento? (Há algum cálculo específico? Existe alguma integração com outros módulos do sistema?)
> 
> **Sugestão de User Story:**
> 
> Como um funcionário do RH, eu preciso lançar a quantidade de aulas semanais ministradas por cada professor no sistema, para que o cálculo da folha de pagamento seja feito corretamente.
> 
> **Próximos passos:**
> 
> Para refinar ainda mais a user story, seria importante obter mais detalhes sobre os pontos listados em "Informações ambíguas/faltantes". Isso ajudará a garantir que a solução desenvolvida atenda às necessidades do usuário e do negócio.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Gerador de User Stories) ---



> Com base no contexto e na análise detalhada, aqui estão algumas user stories adicionais para cobrir diferentes aspectos da necessidade:
> 
> 1.  **Como um funcionário do RH, quero selecionar um professor da lista de funcionários ativos, para registrar a quantidade de aulas semanais ministradas por ele.**
> 2.  **Como um funcionário do RH, quero visualizar o histórico da quantidade de aulas semanais lançadas para um professor, para ter um registro das informações anteriores.**
> 3.  **Como um funcionário do RH, quero que o sistema valide a quantidade de aulas semanais inserida, alertando sobre possíveis inconsistências ou limites excedidos, para evitar erros no lançamento.**
> 4.  **Como um funcionário do RH, quero ter a opção de registrar aulas extras ou substituições de um professor, para que o cálculo da folha de pagamento reflita com precisão as atividades realizadas.**
> 5.  **Como um funcionário do RH, quero confirmar os dados lançados antes de salvar a quantidade de aulas semanais de um professor, para garantir que as informações estejam corretas.**
> 
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Gerador de Critérios de Aceitação) ---



> ## Critérios de Aceitação Detalhados
> 
> Com base nas user stories fornecidas, aqui estão os critérios de aceitação detalhados, focados em clareza, objetividade e testabilidade:
> 
> **User Story 1: Selecionar um professor da lista de funcionários ativos**
> 
> *   O sistema deve exibir uma lista de todos os professores **ativos** no sistema.
> *   A lista de professores deve ser exibida em ordem alfabética (sobrenome, nome).
> *   Deve ser possível filtrar a lista de professores por nome ou ID funcional.
> *   Ao selecionar um professor, o sistema deve exibir os detalhes do professor (nome, ID funcional, disciplina(s) lecionada(s)).
> *   Se não houver professores ativos no sistema, o sistema deve exibir uma mensagem clara informando que "Não há professores ativos cadastrados".
> *   O sistema deve retornar uma mensagem de erro caso o usuário tente selecionar um professor inativo.
> 
> **User Story 2: Visualizar o histórico da quantidade de aulas semanais lançadas para um professor**
> 
> *   Em tela de detalhe do professor, o sistema deve exibir um histórico de todas as quantidades de aulas semanais lançadas para o professor selecionado.
> *   O histórico deve incluir a data do lançamento, a quantidade de aulas e o usuário do RH que realizou o lançamento.
> *   O histórico deve ser exibido em ordem cronológica inversa (do mais recente para o mais antigo).
> *   Deve ser possível filtrar o histórico por período (data de início e data de fim).
> *   Se não houver histórico de lançamentos para o professor, o sistema deve exibir a mensagem "Não há histórico de lançamentos para este professor".
> *   A quantidade de registros exibidos por página deve ser configurável (ex: 10, 20, 50 registros por página).
> 
> **User Story 3: Validar a quantidade de aulas semanais inserida**
> 
> *   O sistema deve validar se o campo "Quantidade de Aulas Semanais" é um número inteiro positivo.
> *   Se o valor inserido não for um número inteiro, o sistema deve exibir uma mensagem de erro clara e informativa (ex: "Por favor, insira um número inteiro válido").
> *   Se o valor inserido for um número negativo, o sistema deve exibir uma mensagem de erro (ex: "A quantidade de aulas semanais não pode ser um número negativo").
> *   O sistema deve validar se a quantidade de aulas semanais inserida não excede o limite máximo permitido (definir qual é o limite máximo).
> *   Se a quantidade de aulas exceder o limite, o sistema deve exibir uma mensagem de erro (ex: "A quantidade de aulas semanais excede o limite máximo permitido de [limite]").
> *   O sistema deve permitir inserir o valor '0' (zero) no campo "Quantidade de Aulas Semanais".
> 
> **User Story 4: Registrar aulas extras ou substituições**
> 
> *   O sistema deve permitir registrar aulas extras ou substituições separadamente das aulas semanais regulares.
> *   Deve haver um campo específico para registrar a quantidade de aulas extras/substituições.
> *   O sistema deve permitir selecionar o tipo de aula extra/substituição (ex: substituição por licença médica, aula extra por projeto específico).
> *   Deve ser possível adicionar uma observação/justificativa para a aula extra/substituição.
> *   As aulas extras/substituições devem ser exibidas separadamente das aulas semanais regulares no histórico.
> *   A validação da quantidade de aulas extras/substituições deve seguir as mesmas regras da quantidade de aulas semanais regulares (inteiro positivo, limite máximo).
> 
> **User Story 5: Confirmar os dados lançados antes de salvar**
> 
> *   Antes de salvar a quantidade de aulas semanais, o sistema deve exibir uma tela de confirmação com todos os dados inseridos (nome do professor, quantidade de aulas semanais, aulas extras/substituições, data do lançamento).
> *   Na tela de confirmação, deve haver um botão "Confirmar" para salvar os dados e um botão "Cancelar" para retornar à tela de edição.
> *   Ao clicar em "Confirmar", o sistema deve exibir uma mensagem de sucesso (ex: "Lançamento realizado com sucesso!").
> *   Ao clicar em "Cancelar", o sistema deve retornar à tela de edição, preservando os dados inseridos.
> *   O sistema deve registrar a ação de lançamento no histórico do professor, incluindo o usuário do RH que realizou o lançamento e a data/hora do lançamento.
> 
> Estes critérios de aceitação detalhados fornecem uma base sólida para o desenvolvimento e teste da funcionalidade. Eles garantem que o sistema atenda às necessidades do usuário e que o cálculo da folha de pagamento seja feito corretamente.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Gerador de Cenários de Teste) ---



> ## Cenários de Teste BDD
> 
> **User Story 1: Selecionar um professor da lista de funcionários ativos**
> 
> ```gherkin
> Cenário: Exibir lista de professores ativos
>     Dado que o sistema possui professores ativos cadastrados
>     Quando o funcionário do RH acessa a tela de lançamento de aulas
>     Então o sistema deve exibir uma lista de todos os professores ativos
> 
> Cenário: Listar professores em ordem alfabética
>     Dado que o sistema possui professores ativos cadastrados
>     Quando o funcionário do RH acessa a tela de lançamento de aulas
>     Então a lista de professores deve ser exibida em ordem alfabética (sobrenome, nome)
> 
> Cenário: Filtrar lista de professores por nome
>     Dado que o sistema possui professores ativos cadastrados
>     Quando o funcionário do RH digita um nome no campo de filtro
>     Então o sistema deve exibir apenas os professores cujo nome corresponda ao filtro
> 
> Cenário: Filtrar lista de professores por ID funcional
>     Dado que o sistema possui professores ativos cadastrados
>     Quando o funcionário do RH digita um ID funcional no campo de filtro
>     Então o sistema deve exibir apenas o professor cujo ID funcional corresponda ao filtro
> 
> Cenário: Exibir detalhes do professor ao selecionar
>     Dado que o funcionário do RH seleciona um professor da lista
>     Quando o sistema exibe os detalhes do professor
>     Então o sistema deve exibir o nome, ID funcional e disciplina(s) lecionada(s) do professor
> 
> Cenário: Exibir mensagem quando não houver professores ativos
>     Dado que não existem professores ativos no sistema
>     Quando o funcionário do RH acessa a tela de lançamento de aulas
>     Então o sistema deve exibir a mensagem "Não há professores ativos cadastrados"
> 
> Cenário: Exibir mensagem de erro ao tentar selecionar um professor inativo
>     Dado que o sistema possui um professor inativo cadastrado
>     Quando o funcionário do RH tenta selecionar o professor inativo
>     Então o sistema deve retornar uma mensagem de erro informando que o professor está inativo
> ```
> 
> **User Story 2: Visualizar o histórico da quantidade de aulas semanais lançadas para um professor**
> 
> ```gherkin
> Cenário: Exibir histórico de lançamentos na tela de detalhe do professor
>     Dado que o professor possui lançamentos de aulas semanais registrados
>     Quando o funcionário do RH acessa a tela de detalhe do professor
>     Então o sistema deve exibir um histórico de todos os lançamentos de aulas semanais
> 
> Cenário: Histórico deve incluir data, quantidade de aulas e usuário do RH
>     Dado que o professor possui lançamentos de aulas semanais registrados
>     Quando o sistema exibe o histórico de lançamentos
>     Então cada registro no histórico deve incluir a data do lançamento, a quantidade de aulas e o usuário do RH que realizou o lançamento
> 
> Cenário: Exibir histórico em ordem cronológica inversa
>     Dado que o professor possui lançamentos de aulas semanais registrados
>     Quando o sistema exibe o histórico de lançamentos
>     Então o histórico deve ser exibido em ordem cronológica inversa (do mais recente para o mais antigo)
> 
> Cenário: Filtrar histórico por período
>     Dado que o professor possui lançamentos de aulas semanais registrados
>     Quando o funcionário do RH define um período para o filtro
>     Então o sistema deve exibir apenas os lançamentos que se encaixam no período definido
> 
> Cenário: Exibir mensagem quando não houver histórico de lançamentos
>     Dado que o professor não possui lançamentos de aulas semanais registrados
>     Quando o funcionário do RH acessa a tela de detalhe do professor
>     Então o sistema deve exibir a mensagem "Não há histórico de lançamentos para este professor"
> 
> Cenário: Exibir quantidade de registros configurável por página
>     Dado que o funcionário do RH deseja visualizar o histórico de lançamentos
>     Quando o sistema exibe o histórico de lançamentos
>     Então a quantidade de registros exibidos por página deve ser configurável (ex: 10, 20, 50 registros por página)
> ```
> 
> **User Story 3: Validar a quantidade de aulas semanais inserida**
> 
> ```gherkin
> Cenário: Validar que o campo "Quantidade de Aulas Semanais" é um número inteiro positivo
>     Dado que o funcionário do RH tenta inserir um valor não numérico no campo "Quantidade de Aulas Semanais"
>     Quando o funcionário do RH salva o lançamento
>     Então o sistema deve exibir uma mensagem de erro "Por favor, insira um número inteiro válido"
> 
> Cenário: Validar que o campo "Quantidade de Aulas Semanais" não é um número negativo
>     Dado que o funcionário do RH tenta inserir um número negativo no campo "Quantidade de Aulas Semanais"
>     Quando o funcionário do RH salva o lançamento
>     Então o sistema deve exibir uma mensagem de erro "A quantidade de aulas semanais não pode ser um número negativo"
> 
> Cenário: Validar que a quantidade de aulas semanais inserida não excede o limite máximo permitido
>     Dado que o limite máximo de aulas semanais é X
>     E o funcionário do RH tenta inserir uma quantidade de aulas semanais maior que X
>     Quando o funcionário do RH salva o lançamento
>     Então o sistema deve exibir uma mensagem de erro "A quantidade de aulas semanais excede o limite máximo permitido de X"
> 
> Cenário: Permitir inserir o valor '0' (zero) no campo "Quantidade de Aulas Semanais"
>     Dado que o funcionário do RH insere o valor '0' no campo "Quantidade de Aulas Semanais"
>     Quando o funcionário do RH salva o lançamento
>     Então o sistema deve salvar o lançamento com sucesso
> ```
> 
> **User Story 4: Registrar aulas extras ou substituições**
> 
> ```gherkin
> Cenário: Permitir registrar aulas extras ou substituições separadamente
>     Dado que o funcionário do RH está lançando aulas
>     Quando o funcionário do RH insere aulas extras ou substituições
>     Então o sistema deve permitir registrar essas aulas separadamente das aulas semanais regulares
> 
> Cenário: Deve haver um campo específico para registrar a quantidade de aulas extras/substituições
>     Dado que o funcionário do RH está lançando aulas extras ou substituições
>     Quando o funcionário do RH insere as informações
>     Então o sistema deve apresentar um campo específico para a quantidade de aulas extras/substituições
> 
> Cenário: Permitir selecionar o tipo de aula extra/substituição
>     Dado que o funcionário do RH está lançando aulas extras ou substituições
>     Quando o funcionário do RH insere as informações
>     Então o sistema deve permitir selecionar o tipo de aula extra/substituição (ex: substituição por licença médica, aula extra por projeto específico)
> 
> Cenário: Permitir adicionar uma observação/justificativa para a aula extra/substituição
>     Dado que o funcionário do RH está lançando aulas extras ou substituições
>     Quando o funcionário do RH insere as informações
>     Então o sistema deve permitir adicionar uma observação/justificativa para a aula extra/substituição
> 
> Cenário: Aulas extras/substituições devem ser exibidas separadamente no histórico
>     Dado que o professor possui aulas extras/substituições lançadas
>     Quando o funcionário do RH visualiza o histórico do professor
>     Então as aulas extras/substituições devem ser exibidas separadamente das aulas semanais regulares
> 
> Cenário: Validação da quantidade de aulas extras/substituições segue as mesmas regras das aulas regulares
>    Dado que o funcionário do RH tenta inserir um valor inválido no campo de aulas extras/substituições
>    Quando o funcionário do RH salva o lançamento
>    Então o sistema deve aplicar as mesmas regras de validação das aulas regulares (inteiro positivo, limite máximo)
> ```
> 
> **User Story 5: Confirmar os dados lançados antes de salvar**
> 
> ```gherkin
> Cenário: Exibir tela de confirmação com todos os dados inseridos
>     Dado que o funcionário do RH preencheu todos os dados do lançamento
>     Quando o funcionário do RH tenta salvar o lançamento
>     Então o sistema deve exibir uma tela de confirmação com o nome do professor, quantidade de aulas semanais, aulas extras/substituições e data do lançamento
> 
> Cenário: Tela de confirmação deve ter botões "Confirmar" e "Cancelar"
>     Dado que o sistema exibe a tela de confirmação
>     Então a tela de confirmação deve ter um botão "Confirmar" para salvar os dados e um botão "Cancelar" para retornar à tela de edição
> 
> Cenário: Exibir mensagem de sucesso ao clicar em "Confirmar"
>     Dado que o funcionário do RH clica no botão "Confirmar"
>     Então o sistema deve exibir uma mensagem de sucesso "Lançamento realizado com sucesso!"
> 
> Cenário: Retornar à tela de edição ao clicar em "Cancelar"
>     Dado que o funcionário do RH clica no botão "Cancelar"
>     Então o sistema deve retornar à tela de edição, preservando os dados inseridos
> 
> Cenário: Registrar a ação de lançamento no histórico do professor
>     Dado que o funcionário do RH confirma o lançamento
>     Quando o sistema salva o lançamento
>     Então o sistema deve registrar a ação de lançamento no histórico do professor, incluindo o usuário do RH que realizou o lançamento e a data/hora do lançamento
> ```


--------------------------------------------------------------
